In [ ]:
def lowpass_filter(column,dict):#column, cutoff, order, sample_rate):
    cutoff = dict['cutoff']
    order = dict['order']  
    rate = dict['rate']
    b,a = signal.butter(order,cutoff/(rate/2),btype='lowpass')
    filtered_channel = signal.filtfilt(b,a,column)
    return (filtered_channel)

def get_psd(df,fs,bin_width):
    
    #fs = len(df)/(df.index[-1]-df.index[0])    
    f, psd = signal.welch(df.to_numpy(), 
                          fs=fs, 
                          nperseg=fs/bin_width,
                          #window='hanning',
                          axis=0
                         )

    df_psd = pd.DataFrame(psd)#,columns=df.columns)
    df_psd.columns
    df_psd['Frequency (Hz)'] = f
    df_psd = df_psd.set_index('Frequency (Hz)')
    
    return df_psd[1:] #drop the first value because it makes the plots look bad and is effectively 0

gps_data = data['locations']  # extra GPS entry
frame=pd.DataFrame(gps_data) # Convert to data frame

frame['elapsed_time'] = frame['tUTC'] - frame['tUTC'][0] # change time values to relative for this run

susp=data['suspension']
df=pd.DataFrame(susp)  # choose suspension entry and create dataframe
df=df[:snip]


#clip zero values/sensor dropouts & generate velocity columns
def process_sensor_data(df,rate):
    size=len(df.index)
    timedata,interval=np.linspace(0,size/rate,num=size,retstep=True, endpoint=False) #create timestamps
    df['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
    df.where(df.fork>0,inplace=True)
    df.where(df.shock>0,inplace=True)
    df.where(df.rearWheel>0,inplace=True)
    df['forkVelocity']=np.gradient(df.fork,df.time)
    df['shockVelocity']=np.gradient(df.shock,df.time)
    df['rearVelocity']=np.gradient(df.rearWheel,df.time)
    return df